In [ ]:
#Import necessary libraries and APIs
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff
import seaborn as sns 

In [ ]:
#Narrow search based on ticker symbol and start and end date of the historical data pull
ticker = 'ESGU'
start = '2017-11-17'
end = '2022-11-01'

In [ ]:
#Download the stock data based on the variables outlined above
stock_data = yf.download(ticker, start, end, adjusted=True)
stock_data.head()

In [ ]:
#Run the Fama French library to pull, Mkt-RF (market risk factor), SMB (performance of small cap stocks relative to large cap) and HML (high book to market value stocks compared to low book to market)
ff_monthly = gff.famaFrench3Factor(frequency='m')
ff_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff_monthly.set_index('Date', inplace=True)
ff_monthly.tail()

In [ ]:
#Looking just at the adjusted close amounts on a per month basis from the Yahoo Finance pull, look at the percent change in comparison to prior month's data, and drop empty data fields
stock_returns = stock_data['Adj Close'].resample('m').last().pct_change().dropna()

#Merge stock details with the Fama French details
stock_returns.name = "Month_Rtn"
ff_data = ff_monthly.merge(stock_returns,on='Date')
ff_data.tail()

In [ ]:
#Define variables required to calculate the beta values
X = ff_data[['Mkt-RF', 'SMB', 'HML']]
y = ff_data['Month_Rtn'] - ff_data['RF']
X = sm.add_constant(X)


In [ ]:
#Develop a model using the variables defined above
ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

In [ ]:
#Redefine values to help calculate the CAPM value
x_capm = ff_data[['Mkt-RF']]
y_capm = ff_data['Month_Rtn'] - ff_data['RF']
X_capm = sm.add_constant(x_capm)


In [ ]:
#Run the CAPM calculation to find the intercept, and corresponding betas
ff_model_capm = sm.OLS(y_capm, X_capm).fit()
print(ff_model_capm.summary())
intercept_capm, b1_capm = ff_model_capm.params

In [ ]:
#Define variables to calculate the expected return using both the CAPM and Fama French
mkt_rf = .61
hml = .38
smb = .25
rf = .33

In [ ]:
#Create and run the CAPM expected return calculation using the variables and inputs from above
capm_exreturn = rf + (mkt_rf*b1_capm)
calc_capm_exreturn = (1+(capm_exreturn/100))**12-1
calc_capm_exreturn

In [ ]:
# Print results
print(f"Using the CAPM model, our expected rate of return on {ticker} is {round(100*calc_capm_exreturn,2)}%.")

In [ ]:
#Create and run the Fama French expected return calculation using the variables and inputs from above
ff_exreturn = rf + (mkt_rf*b1)+(smb*b2)+(hml*b3)
calc_ff_exreturn = (1+(ff_exreturn/100))**12-1
calc_ff_exreturn

In [ ]:
# Print results
print(f"Using the Fama French 3 Factor model, our expected rate of return on {ticker} is {round(100*calc_ff_exreturn,2)}%.")

In [ ]:
#Display a regression of the market return agains the chosen stock
sns.regplot(data = ff_data, x = 'Mkt-RF', y='Month_Rtn' )